In [ ]:
!pip install xgboost --upgrade
!pip install tensorflow --upgrade
!pip install tensorflow_addons

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import VarianceThreshold
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics import roc_curve,auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Input,Activation,Dropout,BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint,EarlyStopping,LearningRateScheduler,ReduceLROnPlateau,TensorBoard
from keras.utils import plot_model
from keras.metrics import AUC
from keras.optimizers import Adam
import datetime
import tensorflow_addons as tfa
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
import shutil

In [4]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/case_study_1/train.csv")
Y     = train['TARGET'].tolist()
train.drop(['ID','TARGET'],axis=1,inplace=True)

In [5]:
test = pd.read_csv("/content/drive/MyDrive/case_study_1/test.csv")

In [6]:
def modelling(X,Y):
  """
  This function returns AUC score using the best model found during the experiments
  """
  with open('/content/drive/MyDrive/case_study_1/experiment_1/X_preprocessed','rb') as file:
    X_preprocessed = pickle.load(file) 
  X = X[list(X_preprocessed.columns)].copy()

  with open('/content/drive/MyDrive/case_study_1/experiment_1/standard_scalar','rb') as file:
    standard_scalar_1 = pickle.load(file)
  x_transformed = standard_scalar_1.transform(X)

  with open('/content/drive/MyDrive/case_study_1/experiment_1/best_model','rb') as file:
    best_model = pickle.load(file)  
  y_pred = best_model.predict_proba(x_transformed)[:,1]

  fpr,tpr,thresholds = roc_curve(Y,y_pred)
  score =  auc(fpr,tpr)

  return score
modelling(train,Y)  

0.8557255502702267

In [7]:
def estimate(test):
  """
  This function returns the predictions using the best model found during the experiments
  """
  with open('/content/drive/MyDrive/case_study_1/experiment_1/X_preprocessed','rb') as file:
    X_preprocessed = pickle.load(file) 
  
  test_topfeatures = test[list(X_preprocessed.columns)].copy()
  with open('/content/drive/MyDrive/case_study_1/experiment_1/standard_scalar','rb') as file:
    standard_scalar_1 = pickle.load(file)
  x_transformed = standard_scalar_1.transform(test_topfeatures)

  with open('/content/drive/MyDrive/case_study_1/experiment_1/best_model','rb') as file:
    best_model = pickle.load(file) 
  y_pred = best_model.predict_proba(x_transformed)[:,1]
  
  return y_pred

In [8]:
estimate(test)

array([0.04552065, 0.04907413, 0.00208657, ..., 0.00349204, 0.05088374,
       0.0015895 ], dtype=float32)